In [1]:
# Import docker library and create client connection
import docker
client = docker.DockerClient(base_url='unix://var/run/docker.sock') 

In [2]:
volumes = {
    '/jukebox/LightSheetData/atlas/neuroglancer/atlas/princetonmouse':{
        'bind':'/mnt/data',
        'mode':'ro'
    },
    '/home/ahoag/Git/nglancer-frame/libraries':{
        'bind':'/opt/libraries'
    }
}



environment = {
    'PYTHONPATH':'/opt/libraries',
    'CONFIGPROXY_AUTH_TOKEN':"'31507a9ddf3e41cf86b58ffede2db68326657437704461ae2c1a4018d55e18f0'"
}

In [3]:
client.containers.get('testcv1_container')

<Container: beae7d7e02>

In [15]:
container = client.containers.run('cloudv_test',
                                  volumes=volumes,
                                  environment=environment,
                                  network='nglancer',
                                  name='mytestname',
                                  detach=True)

In [2]:
import requests
import progproxy as pp

In [3]:
proxy_h = pp.progproxy(target_hname='confproxy')

2020-01-28 12:51:52,482 - progproxy - DEBUG - Programable Proxy Object Created
2020-01-28 12:51:52,484 - progproxy - DEBUG - Target set to http://confproxy:8001
2020-01-28 12:51:52,485 - progproxy - DEBUG - header set to {'Authorization': "token '31507a9ddf3e41cf86b58ffede2db68326657437704461ae2c1a4018d55e18f0'"}


In [6]:
headers = {'Authorization': "token '31507a9ddf3e41cf86b58ffede2db68326657437704461ae2c1a4018d55e18f0'"}
proxy_h.addroute(proxypath='testvol3',proxytarget="http://mytestname:1337")

2020-01-28 12:27:17,201 - progproxy - DEBUG - adding path testvol3 pointing to http://mytestname:1337
2020-01-28 12:27:17,215 - progproxy - DEBUG - path added successfully


In [6]:
response = requests.get('http://confproxy:8000/testvol4/info',headers=header)
response.text

'{\n  "data_type": "uint16",\n  "num_channels": 1,\n  "scales": [\n    {\n      "chunk_sizes": [\n        [\n          1024,\n          1024,\n          1\n        ]\n      ],\n      "encoding": "raw",\n      "key": "20000_20000_20000",\n      "resolution": [\n        20000,\n        20000,\n        20000\n      ],\n      "size": [\n        352,\n        640,\n        540\n      ],\n      "voxel_offset": [\n        0,\n        0,\n        0\n      ]\n    }\n  ],\n  "type": "image"\n}'

In [4]:
header = {'Authorization':"token '31507a9ddf3e41cf86b58ffede2db68326657437704461ae2c1a4018d55e18f0'"}
response = requests.get('http://confproxy:8001/api/routes',headers=header)
print(response)
print(response.text)

<Response [200]>
{"/cloudvols/0f11160e10d8/testvol1":{"target":"http://testcv1_container:1337","last_activity":"2020-01-28T17:49:00.256Z"},"/cloudvols/0f11160e10d8/testvol2":{"target":"http://testcv2_container:1337","last_activity":"2020-01-28T17:49:01.652Z"},"/viewers/0f11160e10d8":{"target":"http://testng_container:8080/","last_activity":"2020-01-28T17:49:03.038Z"}}


In [6]:
import json

In [8]:
routes_dict=json.loads(response.text)
routes_dict

{'/cloudvols/0f11160e10d8/testvol1': {'target': 'http://testcv1_container:1337',
  'last_activity': '2020-01-28T17:49:00.256Z'},
 '/cloudvols/0f11160e10d8/testvol2': {'target': 'http://testcv2_container:1337',
  'last_activity': '2020-01-28T17:49:01.652Z'},
 '/viewers/0f11160e10d8': {'target': 'http://testng_container:8080/',
  'last_activity': '2020-01-28T17:49:03.038Z'}}

In [10]:
routes_dict.keys()

dict_keys(['/cloudvols/0f11160e10d8/testvol1', '/cloudvols/0f11160e10d8/testvol2', '/viewers/0f11160e10d8'])

In [2]:
 ng_environment = {
        'PYTHONPATH':'/opt/libraries',
        'CONFIGPROXY_AUTH_TOKEN':"'31507a9ddf3e41cf86b58ffede2db68326657437704461ae2c1a4018d55e18f0'"
    }
ng_container = client.containers.run('ngdemo',
                                  environment=ng_environment,
                                  network='nglancer',
                                  name='testng_container',
                                  detach=True) 

APIError: 409 Client Error: Conflict ("Conflict. The container name "/testng_container" is already in use by container "39cdbf9f5fa4c66feac1b154bcb68d91641d9c89134a9646c4d7294d5211fc53". You have to remove (or rename) that container to be able to reuse that name.")